In [1]:
import os

for dirname, _, filenames in os.walk(r'C:\AI Practical\Groceries_dataset.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
 
try:
    import apyori
except:
    
    from apyori import apriori

In [3]:
df = pd.read_csv("C:\AI Practical\Groceries_dataset.csv")
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\DELL\AppData\Local\Temp\ipykernel_1392\3026202025.py:1: SyntaxWarning: invalid escape sequence '\A'
  df = pd.read_csv("C:\AI Practical\Groceries_dataset.csv")


,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [4]:
df.isnull().any()

Member_number      False
Date               False
itemDescription    False
dtype: bool

In [5]:
all_products = df['itemDescription'].unique()
print("Total products: {}".format(len(all_products)))

Total products: 167


In [6]:
def distribution_plot(x, y, yaxis="Count", xaxis="Products"):
    fig = go.Figure([go.Bar(x=x, y=y)])
    fig.update_layout(
        title="Top 10 Products by Count",
        xaxis_title=xaxis,
        yaxis_title=yaxis,
        template="plotly_dark"
    )
    fig.show()

In [7]:
x = df['itemDescription'].value_counts()
x = x.sort_values(ascending=False)
x = x[:10]

distribution_plot(x=x.index, y=x.values, yaxis="Count", xaxis="Products")

In [8]:
one_hot = pd.get_dummies(df['itemDescription'])
df.drop('itemDescription', inplace=True, axis=1)
df = df.join(one_hot)
df.head()

,Member_number,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1808,21-07-2015,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2552,05-01-2015,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,2300,19-09-2015,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1187,12-12-2015,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3037,01-02-2015,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [9]:
records = df.groupby(["Member_number", "Date"])[all_products].sum()
records = records.reset_index()[all_products]

In [11]:
def get_Pnames(x):
    for product in all_products:
        if x[product] > 0:
            x[product] = product
    return x

records = records.apply(get_Pnames, axis = 1)
records.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_1392\372275000.py:4: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'whole milk' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\DELL\AppData\Local\Temp\ipykernel_1392\372275000.py:4: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'canned beer' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\DELL\AppData\Local\Temp\ipykernel_1392\372275000.py:4: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sausage' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\DELL\AppData\Local\Temp\ipykernel_1392\372275000.py:4: FutureWarning:

Setting an item of incompatible dtype is

,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,...,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
0,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
print("total traisactions : {}".format(len(records)))

total traisactions : 14963


In [14]:
x = records.values
x = [sub[~(sub == 0)].tolist() for sub in x if sub[sub != 0].tolist()]
transactions = x

In [15]:
transactions[0:10]

[['whole milk', 'yogurt', 'sausage', 'semi-finished bread'],
 ['whole milk', 'pastry', 'salty snack'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['soda', 'pickled vegetables'],
 ['frankfurter', 'curd'],
 ['whole milk', 'rolls/buns', 'sausage'],
 ['whole milk', 'soda'],
 ['beef', 'white bread'],
 ['frankfurter', 'soda', 'whipped/sour cream']]

In [18]:
from apyori import apriori
rules = apriori(transactions, min_support=0.00030, min_confidence=0.05, min_lift=3, min_length=2, target="rules")
association_results = list(rules)

In [19]:
for item in association_results:
    
    pair = item[0]
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])
    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("========================================")    

Rule: fruit/vegetable juice -> liver loaf
Support: 0.00040098910646260775
Confidence: 0.12
Lift: 3.5276227897838903
Rule: pickled vegetables -> ham
Support: 0.0005346521419501437
Confidence: 0.05970149253731344
Lift: 3.4895055970149254
Rule: meat -> roll products 
Support: 0.0003341575887188398
Confidence: 0.06097560975609757
Lift: 3.620547812620984
Rule: salt -> misc. beverages
Support: 0.0003341575887188398
Confidence: 0.05617977528089888
Lift: 3.5619405827461437
Rule: misc. beverages -> spread cheese
Support: 0.0003341575887188398
Confidence: 0.05
Lift: 3.170127118644068
Rule: soups -> seasonal products
Support: 0.0003341575887188398
Confidence: 0.10416666666666667
Lift: 14.704205974842768
Rule: sugar -> spread cheese
Support: 0.00040098910646260775
Confidence: 0.06
Lift: 3.3878490566037733
Rule: bottled beer -> butter
Support: 0.0003341575887188398
Confidence: 0.2
Lift: 3.3140642303433
Rule: bottled beer -> hard cheese
Support: 0.0003341575887188398
Confidence: 0.17857142857142858
